In [1]:
s_fl = 'D:\\H_Strategy\\src\\trd_dates.txt'

In [2]:
fl = open(s_fl, encoding='utf-8')

In [10]:
lsLn = fl.readlines()

In [11]:
fl.close()

In [1]:
from datetime import datetime, timedelta

In [15]:
dt = datetime.today()

dt = dt - timedelta(days = 90)

In [20]:
dtBgn = dt

In [23]:
sBgnDate = dtBgn.strftime('%Y%m%d')

In [26]:
lsTrdDates = [s.strip().replace('-', '') for s in lsLn]

In [29]:
dTrdCal = {}
for i in range(1, len(lsTrdDates)-1):
    sCurDate = lsTrdDates[i]
    if sCurDate < sBgnDate:
        continue
    dtCur = datetime.strptime(sCurDate, '%Y%m%d')
    sNxtTrdDt = lsTrdDates[i+1]
    sPrvTrdDt = lsTrdDates[i-1]
    bIsCurTrd = True
    while sCurDate < sNxtTrdDt:
        dTrdCal[sCurDate] = {'nxt_trd':sNxtTrdDt, 'prv_trd':sPrvTrdDt}
        if bIsCurTrd:
            bIsCurTrd = False
            sPrvTrdDt = sCurDate
        dtCur = dtCur + timedelta(days=1)
        sCurDate = dtCur.strftime('%Y%m%d')
    #print(sCurDate)

In [30]:
dTrdCal

{'20170330': {'nxt_trd': '20170331', 'prv_trd': '20170329'},
 '20170331': {'nxt_trd': '20170405', 'prv_trd': '20170330'},
 '20170401': {'nxt_trd': '20170405', 'prv_trd': '20170331'},
 '20170402': {'nxt_trd': '20170405', 'prv_trd': '20170331'},
 '20170403': {'nxt_trd': '20170405', 'prv_trd': '20170331'},
 '20170404': {'nxt_trd': '20170405', 'prv_trd': '20170331'},
 '20170405': {'nxt_trd': '20170406', 'prv_trd': '20170331'},
 '20170406': {'nxt_trd': '20170407', 'prv_trd': '20170405'},
 '20170407': {'nxt_trd': '20170410', 'prv_trd': '20170406'},
 '20170408': {'nxt_trd': '20170410', 'prv_trd': '20170407'},
 '20170409': {'nxt_trd': '20170410', 'prv_trd': '20170407'},
 '20170410': {'nxt_trd': '20170411', 'prv_trd': '20170407'},
 '20170411': {'nxt_trd': '20170412', 'prv_trd': '20170410'},
 '20170412': {'nxt_trd': '20170413', 'prv_trd': '20170411'},
 '20170413': {'nxt_trd': '20170414', 'prv_trd': '20170412'},
 '20170414': {'nxt_trd': '20170417', 'prv_trd': '20170413'},
 '20170415': {'nxt_trd':

In [34]:
# Estimate next year's trade calendar
import lunardate
from lunardate import LunarDate

In [34]:
def genNextYearEstCal():
    # http://www.17jiaoyu.com/bangzhu/kaixueshijian/201612/20161202145339_297507.html
    # https://publicholidays.cn/zh/2018-dates/
    lsHoliday = [
        ('20180101', '20180101'),
        ('20180215', '20180221'),
        ('20180405', '20180406'),
        ('20180430', '20180501'),
        ('20180618', '20180618'),
        ('20180924', '20180924'),
        ('20181001', '20181007')
    ]


    iBgnYr = 2017
    dtBgn = datetime(iBgnYr, 12, 1)
    dtEnd = datetime(iBgnYr+1, 12, 31)

    dTrdCal = {}
    dtCur = dtBgn
    while dtCur <= dtEnd:
        sCur = dtCur.strftime('%Y%m%d')
        bCanTrd = True
        # Python starts with 0 = Monday. 5 = Saturday, 6 = Sunday
        if dtCur.weekday() in (5, 6):
            bCanTrd = False
        else:
            for sHldBgn, sHldEnd in lsHoliday:
                if sCur >= sHldBgn and sCur <= sHldEnd:
                    bCanTrd = False
        dTrdCal[sCur] = {'can_trd': bCanTrd}
        dtCur = dtCur + timedelta(days = 1)
    #end while
    return dTrdCal
#end def

In [35]:
dTrdCal = genNextYearEstCal()

In [12]:
def readDateInfo(sTrdDayFile = 'C:\\workspace\\python\\PriceCompare\\tradeday.txt'):
        
    flTrdDay = open(sTrdDayFile, 'r', encoding='utf-8', errors = 'ignore')

    dTradeDay = {}
    for line in flTrdDay:
        lsCol    = line.strip().split(';')
        sDay     = lsCol[0]
        sWorkDay = lsCol[1]
        sPrevDay = lsCol[2]
        #if sDay >= self.sToday: break
        dTradeDay[sDay] = (sWorkDay, sPrevDay)
    #end for

    flTrdDay.close()

    #print('Previous trade day is %s' % sPrevDay)

    return dTradeDay
#end of def

In [13]:
dTradeDay = readDateInfo()

In [30]:
dCal1 = {}
for sCur, t in dTradeDay.items():
    sCanTrd = True
    if '非' in t[0]:
        sCanTrd = False
    dCal1[sCur] = {'can_trd': sCanTrd}
    '''
    sPrv = t[1]
    d = {'can_trd': sCanTrd, 'prv_trd': sPrv}
    if sCur in dCal1:
        dCal1[sCur].update(d)
    else:
        dCal1[sCur] = d
    if sPrv in dCal1:
        d = dCal1[sPrv]
        if 'nxt_trd' in d:
            if sCur > d['nxt_trd']:
                d['nxt_trd'] = sCur
        else:
            d['nxt_trd'] = sCur
    else:
        dCal1[sPrv] = {'nxt_trd': sCur}
    '''


In [36]:
dCal1.update(dTrdCal)
dTrdCal = dCal1

In [27]:
for k, v in dTrdCal.items():
    if v == 'yes':
        dTrdCal[k] = True
    else:
        dTrdCal[k] = False

In [26]:
dtBgn = datetime.today() - timedelta(days = 90)
sBgn = dtBgn.strftime('%Y%m%d')

In [37]:
lsDates = sorted(dTrdCal.keys())

for i in range(1, len(lsDates)-1):
    sCur = lsDates[i]
    if sCur < sBgn:
        continue
    dtCur = datetime.strptime(sCur, '%Y%m%d')
    for j in range(i+1, len(lsDates)):
        sNxt = lsDates[j]
        if dTrdCal[sNxt]['can_trd']:
            dTrdCal[sCur]['nxt_trd'] = sNxt
            break
    for j in range(i-1, -1, -1):
        sPrv = lsDates[j]
        if dTrdCal[sPrv]['can_trd']:
            dTrdCal[sCur]['prv_trd'] = sPrv
            break
#end for

In [32]:
lunardate.LunarDate.today()

LunarDate(2017, 6, 5, 0)

In [36]:
LunarDate(2018,1,1).toSolarDate()

datetime.date(2018, 2, 16)

In [38]:
dTrdCal

{'20161101': {'can_trd': True},
 '20161102': {'can_trd': True},
 '20161103': {'can_trd': True},
 '20161104': {'can_trd': True},
 '20161105': {'can_trd': False},
 '20161106': {'can_trd': False},
 '20161107': {'can_trd': True},
 '20161108': {'can_trd': True},
 '20161109': {'can_trd': True},
 '20161110': {'can_trd': True},
 '20161111': {'can_trd': True},
 '20161112': {'can_trd': False},
 '20161113': {'can_trd': False},
 '20161114': {'can_trd': True},
 '20161115': {'can_trd': True},
 '20161116': {'can_trd': True},
 '20161117': {'can_trd': True},
 '20161118': {'can_trd': True},
 '20161119': {'can_trd': False},
 '20161120': {'can_trd': False},
 '20161121': {'can_trd': True},
 '20161122': {'can_trd': True},
 '20161123': {'can_trd': True},
 '20161124': {'can_trd': True},
 '20161125': {'can_trd': True},
 '20161126': {'can_trd': False},
 '20161127': {'can_trd': False},
 '20161128': {'can_trd': True},
 '20161129': {'can_trd': True},
 '20161130': {'can_trd': True},
 '20161201': {'can_trd': True},


In [43]:
from pprint import saferepr
def writeTupleDict(d, sFileName, sEncoding = 'utf-8', bReverse = False):
    ls = sorted(d.items(), key = lambda t:t[0], reverse = bReverse)
    if sEncoding == '':
        fl = open(sFileName, 'w', errors = 'ignore')
    else:
        fl = open(sFileName, 'w', encoding = sEncoding, errors = 'ignore')
    for item in ls:
        #sLine = repr(item)
        sLine = saferepr(item)
        print(sLine, file = fl)
    #end for
    fl.close()
#def

In [44]:
writeTupleDict(dTrdCal, 'C:\\workspace\\python\\TradeBot\\params\\TradeCalendar.txt')